In [14]:
# Importing all needed functions
import os
import requests
from bs4 import BeautifulSoup
from astropy.io import fits
import glob
from Calculating_det_angles import estimate_source_angles_detectors  # Importing ma'am's function
from Tools import tools
import threading
from queue import Queue
import pandas as pd
import numpy as np
from keras.models import load_model
from pathlib import Path
import matplotlib.pyplot as plt
import traceback
from numba import jit
from line_profiler import profile
%load_ext line_profiler
from scipy import stats
import pwkit.bblocks

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [15]:
@jit(nopython=True)
def filter_counts(times, channels, min_ch, max_ch, trigtime):
    """
    Filter count data based on channel range and subtract trigtime.
    
    :param times: Array of event times
    :param channels: Array of channel numbers
    :param min_ch: Minimum channel number
    :param max_ch: Maximum channel number
    :param trigtime: Trigger time to be subtracted
    :return: Filtered and adjusted count data
    """
    counts = []
    for t, ch in zip(times, channels):
        if min_ch <= ch <= max_ch:
            counts.append(t - trigtime)
    return np.array(counts)

@profile
def process_folder(folder, bin_list,data_no,r,chr1,chr2,data_set_path,source_data_set_path, plot = False):
    """
    Process a single folder of data.
    
    :param folder: Name of the folder to process
    :param bin_list: List of bin sizes to use
    """
    try:
        # Extract event information from folder name
        event_type, event = folder.split("_")

        # if event_type != "SGR":
        #     return
        
        # # check if the file is processed
        # data_file_path = Path(data_set_path) / f"{event_type}_{event}"
        # if data_file_path.exists():
        #     # print(f"File already processed: {data_file_path}")
        #     return

        # Construct file pattern and find matching files
        file_pattern = str(Path(source_data_set_path) / folder / 'current' / "*_tte_*")
        NaI_detector = glob.glob(file_pattern)

        # Read data from FITS file
        with fits.open(NaI_detector[0], memmap=True) as hdul:
            all_count_data = hdul[2].data
            trigtime = float(hdul[2].header['TRIGTIME'])

        # Extract times and channels from all_count_data
        times = all_count_data['TIME'].astype(float)
        channels = all_count_data['PHA'].astype(int)

        # Filter counts for different channel ranges
        counts_1 = filter_counts(times, channels, chr1[0], chr1[1], trigtime)
        counts_2 = filter_counts(times, channels, chr2[0], chr2[1], trigtime)

        data_array = []
        
        # Process both count ranges
        for i in bin_list:
            sep_sig = []
            ch_no = 1
            for counts in [counts_1,counts_2]:
                # Calculate range and bin size
                range_min = -data_no * i * r
                range_max = data_no * i * (1-r)
                bin_size = i

                # Create histogram
                bin_edges = np.arange(range_min, range_max, bin_size)
                hist, edges = np.histogram(counts, bins=bin_edges)
                hist = adjust_zeros(hist)
                bin_widths = edges[1:]-edges[:-1]
                hrates = hist / bin_size

                # Perform Bayesian Block Analysis
                bayes = pwkit.bblocks.bin_bblock(widths = bin_widths, counts = hist, p0 = 0.01)

                # Getting the results of the analysis
                block_starts = (bayes.get('blockstarts') * i) + range_min
                block_stops = block_starts + bayes.get('widths')
                bins = np.array(list(block_starts) + list([block_stops[-1]]))
                rates = bayes.get('rates')

                # Find indices where rates are non-zero
                non_zero_indices = np.nonzero(rates)[0]
                
                # Select rates that are non-zero
                rates = rates[non_zero_indices]
                
                # Select corresponding bins
                bins = np.concatenate([bins[non_zero_indices], [bins[non_zero_indices[-1] + 1]]])

                sigw = bins[-2] - bins[1]

                bk2 = bins[1] - (5 * bin_size)
                bk3 = bins[-2] + (5 * bin_size)
                bk1 = bk2 - sigw - (50 * bin_size) if (bk2 - sigw - (50 * bin_size)) > bins[0] else bins[0]
                bk4 = bk3 + sigw + (50 * bin_size) if (bk3 + sigw + (50 * bin_size)) < bins[-1] else bins[-1]

                bkgd = [(bk1,bk2),(bk3,bk4)]

                # Check if background is proper
                bkgd_check = bk1<bk2<bk3<bk4

                # Checking for signal
                # Checks False alarm probability p0, number of bins, and if background is detected around trigger
                if bayes.get('finalp0') >= 0.01 or len(bins)<4 or not(bk2 - 5 <= 0 <= bk3 + 5) or not bkgd_check:
                    if plot:
                        print('no signal')
                        plt.stairs(hrates, edges)
                        plt.show()
                        plt.stairs(rates, bins)
                        plt.show()
                    signal = np.zeros_like(hrates) # send no signal
                                    
                else:
                    y = hrates
                    x = (edges[1:] + edges[:-1]) / 2
                    bkgd_pol,order = best_fit_polynomial(x, y, bkgd)

                    # Create mask for y between bk2 and bk3
                    mask = (x >= bk2) & (x <= bk3)

                    # Apply the mask and take the maximum of 0 and y - bkgd_pol(x)
                    signal = np.zeros_like(y)
                    signal[mask] = np.maximum(0, y[mask] - bkgd_pol(x[mask]))
                    
                    if plot:
                        # Plot the results
                        plt.stairs(hrates,edges)
                        plt.xlabel('Time (s)')
                        plt.ylabel('rates')
                        plt.title(f'light curve ; ch = {ch_no} ;bin size = {bin_size}s')
                        # Plot each shaded area
                        for i, (x1, x2) in enumerate(bkgd):
                            plt.axvspan(x1, x2, alpha=0.3, color='red',label = 'background selection') 
                        # make sure most things are visible
                        if min(edges) < counts.min():
                            plt.gca().set_xlim(left=counts.min())
                        if max(edges) > counts.max():
                            plt.gca().set_xlim(right=counts.max())
                        plt.axvline(x=0, color='red', linestyle='--', linewidth=0.5)
                        # Evaluate the background polynomial for plotting
                        bkgd_y = bkgd_pol(x)
                        
                        # Check if the background blows up
                        if np.any(np.abs(bkgd_y) > 10 * np.max(np.abs(y))) or np.any(np.abs(bkgd_y) < 0.1 * np.min(np.abs(y[y != 0]))):                
                            # Plot the background within the limits
                            mask = (bkgd_y >= plt.ylim()[0]) & (bkgd_y <= plt.ylim()[1])
                            plt.plot(x[mask], bkgd_y[mask], '--', label='background')
                        else:
                            # If it doesn't blow up, plot normally
                            plt.plot(x, bkgd_y, '--', label='background')
                        plt.show()

                        # signal plot
                        plt.title(f'light curve ; ch = {ch_no} ;bin size = {bin_size}s')
                        plt.axvline(x=0, color='red', linestyle='--', linewidth=0.5)
                        plt.xlabel('Time (s)')
                        plt.ylabel('rates')
                        plt.plot(x,signal)
                        plt.show()

                # updatae channel number and save signal
                ch_no = ch_no + 1               
                sep_sig.extend(signal)
                

            # Accumulate histogram data
            data_array.extend(sep_sig)

        # Convert accumulated data to numpy array
        data_array = np.array(data_array)
        
        if plot:
            print('shape', data_array.shape)

        return data_array

    except Exception as e:
        # Handle and log any errors
        print(f'error {e} in {folder}')
             
def adjust_zeros(arr):
    # Create a copy of the input array
    result = np.copy(arr)
    
    # Find first non-zero element
    first_non_zero = np.argmax(arr != 0)
    
    # Find last non-zero element
    last_non_zero = len(arr) - np.argmax(arr[::-1] != 0) - 1
    
    # Replace value after leading zeros with zero if there are leading zeros
    if first_non_zero > 0:
        result[first_non_zero] = 0
    
    # Replace value before trailing zeros with zero if there are trailing zeros
    if last_non_zero < len(arr) - 1:
        result[last_non_zero] = 0
    
    return result

def best_fit_polynomial(x, y, background_intervals):
    # Filter data to include only background intervals
    mask = np.zeros(len(x), dtype=bool)
    for start, end in background_intervals:
        mask |= (x >= start) & (x <= end)
    
    x_bg = x[mask]
    y_bg = y[mask]
    
    best_order = 0
    best_polynomial = None
    best_r_squared = -np.inf
    
    for order in range(1, 5):  # Test polynomials of order 1 to 4
        coeffs = np.polyfit(x_bg, y_bg, order)
        p = np.poly1d(coeffs)
        
        # Calculate R-squared
        y_pred = p(x_bg)
        r_squared = 1 - np.sum((y_bg - y_pred)**2) / np.sum((y_bg - np.mean(y_bg))**2)
        
        # Perform F-test to compare with lower order model
        if order > 1:
            f_statistic = ((r_squared - prev_r_squared) / (order - prev_order)) / \
                          ((1 - r_squared) / (len(x_bg) - order - 1))
            p_value = 1 - stats.f.cdf(f_statistic, 1, len(x_bg) - order - 1)
            
            if p_value < 0.05 and r_squared > best_r_squared:
                best_order = order
                best_polynomial = p
                best_r_squared = r_squared
        else:
            best_order = order
            best_polynomial = p
            best_r_squared = r_squared
        
        prev_order = order
        prev_r_squared = r_squared
    
    return best_polynomial, best_order

def get_flags(y_true , y_pred):    
    y_pred = np.array(y_pred)
    max_index, max_value = tools.list_max(y_pred)

    if np.all(y_pred < 0.3):
        return True

    if y_true.index(max(y_true)) == max_index and max_value >= 0.7:
        return False
    elif max_value > 0.9:
        return True
    else:
        return False  

In [16]:
# Load the model
model_path = r"C:\Users\arpan\OneDrive\Desktop\m17-07-1.keras"
model = load_model(model_path)

In [17]:
# Print model summary
model.summary()

Model: "functional_37"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ 0.001 (InputLayer)  │ (None, 499, 2)    │       0 │ -                    │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ 0.005 (InputLayer)  │ (None, 499, 2)    │       0 │ -                    │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ 0.01 (InputLayer)   │ (None, 499, 2)    │       0 │ -                    │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ 0.1 (InputLayer)    │ (None, 499, 2)    │       0 │ -                    │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ 0.5 (InputLayer)    │ (None, 499, 2)    │       0 │ -                    │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ 1 (InputLayer)      │ (None, 499, 2)    │       0 │ -                    │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ 5 (InputLayer)      │ (None, 499, 2)    │       0 │ -                    │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv1d_126 (Conv1D) │ (None, 484, 4)    │     132 │ 0.001[0][0]          │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv1d_127 (Conv1D) │ (None, 484, 4)    │     132 │ 0.005[0][0]          │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv1d_128 (Conv1D) │ (None, 484, 4)    │     132 │ 0.01[0][0]           │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv1d_129 (Conv1D) │ (None, 484, 4)    │     132 │ 0.1[0][0]            │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv1d_130 (Conv1D) │ (None, 484, 4)    │     132 │ 0.5[0][0]            │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv1d_131 (Conv1D) │ (None, 484, 4)    │     132 │ 1[0][0]              │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv1d_132 (Conv1D) │ (None, 484, 4)    │     132 │ 5[0][0]              │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ flatten_144         │ (None, 1936)      │       0 │ conv1d_126[0][0]     │
│ (Flatten)           │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ flatten_145         │ (None, 1936)      │       0 │ conv1d_127[0][0]     │
│ (Flatten)           │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ flatten_146         │ (None, 1936)      │       0 │ conv1d_128[0][0]     │
│ (Flatten)           │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ flatten_147         │ (None, 1936)      │       0 │ conv1d_129[0][0]     │
│ (Flatten)           │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ flatten_148         │ (None, 1936)      │       0 │ conv1d_130[0][0]     │
│ (Flatten)           │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ flatten_149         │ (None, 1936)      │       0 │ conv1d_131[0][0]     │
│ (Flatten)           │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ flatten_150         │ (None, 1936)      │       0 │ conv1d_132[0][0]     │
│ (Flatten)           │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼────────────────────

 Total params: 27,924,194 (106.52 MB)

 Trainable params: 9,308,064 (35.51 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 18,616,130 (71.01 MB)

In [18]:
# Function to process a single event
def process_event(event_name, event_type,thread_num):

    true_labels = {'GRB': [1,0,0,0],'TGF': [0,1,0,0],'SGR': [0,0,1,0],'SFLARE': [0,0,0,1]}
    dir_path = tools.json_path(r'data_path.json')

    # list of bin sizes
    bin_list = [0.001, 0.005, 0.01, 0.1, 0.5, 1, 5] #[0.001,0.01, 0.1, 1, 5]

    # number of datapoints in a light curve
    data_no = 500

    # ratio of pre-trigger to post-trigger
    r = 0.25

    # channel range
    chr1 = [3, 50]
    chr2 = [51, 124]
    chrn = 2
    dno = data_no - 1
    t = 'n/a'

    try:
        # getting the true label of the event
        if event_type in true_labels.keys():
            y_true = true_labels[event_type]
        elif event_type == 'UNCERT' or event_type == 'UNRELOC' or event_type == 'BELOWHZ':
            y_true = [0,0,0,0]
        else:
            # returning if the event is not GRB, TGF, SGR, or SFLARE
            return 
        
        # creating a temporary folder to store the files
        temp_thread = r'temp'+ str(thread_num)
        temp_path = os.path.join(dir_path, temp_thread)
        tools.create_folder(temp_path)
        event = event_name
        year = '20' + event[2:4] + "/"

        check_url =  'https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/' + year + event + '/current/'

        # Send a GET request to the URL
        response = requests.get(check_url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Find all links in the page
            links = soup.find_all('a')
            
            # Check for .rsp files
            rsp_files = [link.get('href') for link in links if link.get('href', '').endswith('.rsp')]
        
        if len(rsp_files) < 1:
            return

        # URL of the file you want to download
        url = 'wget -q -nH --no-check-certificate --cut-dirs=7 -r -l0 -c -N -np -A "*_trigdat_*" -R "index*" -erobots=off --retr-symlinks https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/' + year + event + '/current/'
        tools.run_wget_download(url, temp_path)

        # Finding Trigdat file
        trig_string = "_trigdat_"
        trig_pattern = os.path.join(temp_path, 'current', f"*{trig_string}*")
        trigdat_file = glob.glob(trig_pattern)

        # Get the spacecraft pointing from here
        event_filename = trigdat_file[0]

        # Getting the RA and DEC
        with fits.open(event_filename, memmap=True) as pha_list:
            ra_obj, dec_obj = (pha_list[0].header['RA_OBJ']), (pha_list[0].header['DEC_OBJ'])

        brightest_nai, bright_nais, brightest_bgo = estimate_source_angles_detectors.angle_to_grb(ra_obj, dec_obj,event_filename)

        for download_nai in  [brightest_nai]:
            # URL of the tte file to download
            url = 'wget -q -nH --no-check-certificate --cut-dirs=7 -r -l0 -c -N -np -A "*_tte_' + download_nai + '_*" -R "index*" -erobots=off --retr-symlinks https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/' + year + event + '/current/'

            # Construct the wget command
            tools.run_wget_download(url, os.path.join(temp_path, event_type + '_' + event))

        data = process_folder(os.path.join(temp_path, event_type + '_' + event),bin_list,data_no,r,chr1,chr2,data_set_path = temp_path,source_data_set_path = temp_path)
        
        X = {str(key): [] for key in bin_list}

        f = 0
        for k in X.keys():
            datlist = []
            for i in range(chrn):
                if t == 'n/a':
                    datlist.append(data[f:f+dno])
                    f = f + int(dno)
                else:
                    datlist.append(data[f:f+int(t/float(k)) - 1])
                    f = f + int(t/float(k)) - 1

            X[k].append(np.array(datlist).transpose())
        
        x={}

        for key in X.keys():
            arr = np.array(X[key][0])
            arr = arr.reshape(1,arr.shape[0],2)
            x[key] = arr
            
        y_pred = model.predict(x)
        y_pred = y_pred[0]
        
        # finding if the y_pred is interesting
        # print(get_flags(y_true, y_pred))
        if get_flags(y_true, y_pred):
            interesting_events.append(event_name)
            print('interesting event')
            print(event_name)
            print(y_true)
            print(y_pred)
            print('-----------------------------')
        elif y_pred[0] > 0.9:
            GRB_candidates.append(event_name)
            print('GRB candidate')
            print(event_name)
            print(y_true)
            print(y_pred)
            print('-----------------------------')
        else:
            print('event')
            print(event_name)
            print(y_true)
            print(y_pred)
            print('-----------------------------')

        # saving the outputs
        output_event_names.append(event_name)
        output_event_types.append(event_type)
        output_event_preds.append(y_pred)
        output_event_trues.append(y_true)

        # Print the size of the queue
        print(event_queue.qsize()) 

    except Exception as e:
        traceback.print_exc()
        print(f"Error processing event {event_name}: {e}")



In [19]:
# Create a queue to store the events
event_queue = Queue()

import warnings

warnings.filterwarnings("ignore", category=RuntimeWarning)

# Function to run the worker threads
def worker(thread_num):
    print(f"Worker thread {thread_num} started.")
    while True:
        # Get an event from the queue
        event_name, event_type = event_queue.get()

        # Process the event
        process_event(event_name, event_type,thread_num)

        # Signal that the task is completed
        event_queue.task_done()
        print(f"Worker thread {thread_num} processed event {event_name}.")

        if event_queue.qsize() == 0:
            break
    
    print(f"Worker thread {thread_num} finished.")

# getting the candidates
df = pd.read_csv('less_than_0.5.csv')

# Assuming 'df' is your DataFrame and 'column_name' is the name of the column
event_names = df['event_name'].values.tolist()
event_types = df['event_type'].values.tolist()
reliability = df['reliability'].values.tolist()

# intializing the outputs
output_event_names = []
output_event_types = []
output_event_preds = []
output_event_trues = []
interesting_events = []
GRB_candidates = []

# Create and start the worker threads
num_threads = 4
threads = []
for i in range(num_threads):
    worker_thread = threading.Thread(target=worker, args=(i,), daemon=True)
    worker_thread.start()
    threads.append(worker_thread)

# Add events to the queue
for event_name, event_type in zip(event_names[:], event_types[:]):
    event_queue.put((event_name, event_type))

# Wait for all events to be processed
event_queue.join()
print("All events have been processed.")

print('done')

Worker thread 0 started.
Worker thread 1 started.
Worker thread 2 started.
Worker thread 3 started.
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\temp0' already exists.
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\temp3' already exists.
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\temp1' already exists.
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\temp2' already exists.
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\temp1' and its contents removed successfully.
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\temp0' and its contents removed successfully.
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\temp3' and its contents removed successfully.
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\temp1' created successfully.
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\temp0' created successfully.
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\temp3' created successfully.
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\temp2' and its content

Traceback (most recent call last):
  File "C:\Users\arpan\AppData\Local\Temp\ipykernel_10316\660161319.py", line 93, in process_event
    datlist.append(data[f:f+dno])
TypeError: 'NoneType' object is not subscriptable


download start
Downloaded bn081225257_tte_n1_ to C:\Users\arpan\OneDrive\Documents\GRB\data\temp3\GRB_bn081225257
Downloaded bn090122694_tte_n0_ to C:\Users\arpan\OneDrive\Documents\GRB\data\temp0\SGR_bn090122694
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
event
bn081225257
[1, 0, 0, 0]
[0.5668328  0.00306125 0.01121196 0.41889402]
-----------------------------
31
Worker thread 3 processed event bn081225257.
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\temp3' already exists.
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\temp3' and its contents removed successfully.
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\temp3' created successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
event
bn090122694
[0, 0, 1, 0]
[8.4586843e-21 3.6629268e-32 1.0000000e+00 4.5792010e-26]
-----------------------------
30
Worker thread 0 processed event bn090122694.
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\temp0' already exists.
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\temp0' and i

In [20]:
def create_dataframe_from_lists(*args, columns=None):
    # Check if any lists were provided
    if not args:
        raise ValueError("No lists provided. Please provide at least one list.")
    
    # Check if all inputs are lists and have the same length
    lengths = set(len(arg) for arg in args)
    if len(lengths) != 1:
        raise ValueError("All input lists must have the same length.")
    
    # Create the DataFrame
    df = pd.DataFrame(list(zip(*args)), columns=columns)
    
    return df

df = create_dataframe_from_lists(output_event_names,
                                 output_event_types,
                                 output_event_preds,
                                 output_event_trues)

df.head()

,0,1,2,3
0,bn080719529,GRB,"[0.9980566, 0.00037892972, 0.00018921478, 0.00...","[1, 0, 0, 0]"
1,bn090328545,SGR,"[2.0986443e-29, 1.632884e-32, 1.0, 7.893495e-32]","[0, 0, 1, 0]"
2,bn090131590,SGR,"[7.12835e-17, 1.8403805e-18, 1.0, 2.1624692e-17]","[0, 0, 1, 0]"
3,bn080815917,GRB,"[0.9999528, 1.0736917e-07, 4.6686524e-05, 4.14...","[1, 0, 0, 0]"
4,bn090122451,SGR,"[0.0, 0.0, 1.0, 0.0]","[0, 0, 1, 0]"


In [24]:
print(df)

              0    1                                                  2  \
0   bn080719529  GRB  [0.9980566, 0.00037892972, 0.00018921478, 0.00...   
1   bn090328545  SGR   [2.0986443e-29, 1.632884e-32, 1.0, 7.893495e-32]   
2   bn090131590  SGR   [7.12835e-17, 1.8403805e-18, 1.0, 2.1624692e-17]   
3   bn080815917  GRB  [0.9999528, 1.0736917e-07, 4.6686524e-05, 4.14...   
4   bn090122451  SGR                               [0.0, 0.0, 1.0, 0.0]   
5   bn090208340  SGR                               [0.0, 0.0, 1.0, 0.0]   
6   bn090206897  SGR                               [0.0, 0.0, 1.0, 0.0]   
7   bn080924766  GRB   [1.0, 5.8434273e-12, 2.133128e-11, 7.553709e-09]   
8   bn090326625  SGR  [7.444337e-23, 2.8494333e-28, 1.0, 2.0693698e-25]   
9   bn080912360  GRB  [0.9995598, 6.214204e-06, 3.4024302e-05, 0.000...   
10  bn090210310  SGR                               [0.0, 0.0, 1.0, 0.0]   
11  bn200604778  GRB  [0.9999808, 4.7067894e-10, 1.0867592e-08, 1.91...   
12  bn081225257  GRB  [0.

In [33]:
count = 0
for index, row in df.iterrows():
    pred = np.round(np.array(row[2]),2)
    true = np.array(row[3])
    if pred.argmax() != true.argmax():
        print(row[0],row[1],pred,true)
        count += 1

print(count)

0


In [25]:
df_fil = df[df[2].argmax() != df[3].argmax()]

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [21]:
print(gg)

NameError: name 'gg' is not defined

In [ ]:
filename = str(model_path).split('/')[-1] + "_uncert_results"

df.to_csv(filename, index=False)
print(f"DataFrame saved to {filename}")

DataFrame saved to C:\Users\arpan\OneDrive\Desktop\m17-07-1.keras_uncert_results


In [22]:
print(GRB_candidates) # ['bn110102432', 'bn150101576', 'bn100729415', 'bn150711543', 'bn150707234', 'bn150914103', 'bn170815526'] [0.9720139  0.0021333  0.00373149 0.02212139]  [0.94786453 0.0044799  0.00902045 0.03863509]

['bn080719529', 'bn080815917', 'bn080924766', 'bn080912360', 'bn200604778', 'bn080928628', 'bn080805496', 'bn080823363', 'bn230320150', 'bn090409288', 'bn181212693', 'bn180804765', 'bn080829790', 'bn081003644', 'bn081017474', 'bn190817953']


In [23]:
print(interesting_events) # ['bn100530292', 'bn211217424', 'bn210403738', 'bn120813978', 'bn120502856', 'bn240320700', 'bn210425089', 'bn231225841', 'bn171003256', 'bn150316434', 'bn230623728', 'bn171202587', 'bn140330899', 'bn220906529', 'bn190326747', 'bn090523133', 'bn160116302', 'bn120312392', 'bn230912087', 'bn190919899', 'bn120725321', 'bn161111083', 'bn131117611', 'bn140504334', 'bn181010392', 'bn240130149', 'bn210818917', 'bn131101050', 'bn221104472', 'bn170712214', 'bn230613572', 'bn151015141', 'bn221214088', 'bn180827314', 'bn090209200', 'bn140513553', 'bn190620579', 'bn240308556', 'bn220619154', 'bn221011605', 'bn220808197', 'bn230217875', 'bn170603580', 'bn151002067', 'bn090818453', 'bn080922709', 'bn221023281', 'bn210504483', 'bn180904763', 'bn220222492', 'bn240206567', 'bn181209226', 'bn210910456', 'bn140828673', 'bn121015799', 'bn160523504', 'bn090602780', 'bn140619498', 'bn120204771', 'bn130424231', 'bn100322678', 'bn180406562']

[]


In [ ]:
# GRB_candidates = ['bn110102432', 'bn150101576', 'bn100729415', 'bn150711543', 'bn150707234', 'bn150914103', 'bn170815526']
# interesting_events = ['bn100530292', 'bn211217424', 'bn210403738', 'bn120813978', 'bn120502856', 'bn240320700', 'bn210425089', 'bn231225841', 'bn171003256', 'bn150316434', 'bn230623728', 'bn171202587', 'bn140330899', 'bn220906529', 'bn190326747', 'bn090523133', 'bn160116302', 'bn120312392', 'bn230912087', 'bn190919899', 'bn120725321', 'bn161111083', 'bn131117611', 'bn140504334', 'bn181010392', 'bn240130149', 'bn210818917', 'bn131101050', 'bn221104472', 'bn170712214', 'bn230613572', 'bn151015141', 'bn221214088', 'bn180827314', 'bn090209200', 'bn140513553', 'bn190620579', 'bn240308556', 'bn220619154', 'bn221011605', 'bn220808197', 'bn230217875', 'bn170603580', 'bn151002067', 'bn090818453', 'bn080922709', 'bn221023281', 'bn210504483', 'bn180904763', 'bn220222492', 'bn240206567', 'bn181209226', 'bn210910456', 'bn140828673', 'bn121015799', 'bn160523504', 'bn090602780', 'bn140619498', 'bn120204771', 'bn130424231', 'bn100322678', 'bn180406562']

for i in GRB_candidates:
    print(True if i in interesting_events else False)
    if i in interesting_events:
        print(i)
        

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [ ]:
for i in GRB_candidates:
    n = output_event_names.index(i)
    print(output_event_names[n], output_event_types[n], output_event_preds[n])

bn110222659 UNCERT [1.0000000e+00 2.0652371e-14 7.7315329e-15 2.4463487e-14]
bn240323372 UNCERT [9.9237108e-01 4.9943681e-04 1.4001724e-03 5.7293461e-03]
bn090407780 UNCERT [1.0000000e+00 2.8554006e-10 1.6803917e-09 9.8042419e-09]
bn131016664 UNCERT [1.0000000e+00 1.6849875e-13 6.8798214e-15 4.0638125e-13]
bn110102432 UNCERT [9.9977177e-01 2.7596350e-09 6.6443114e-08 2.2813222e-04]
bn240628601 UNCERT [9.2004341e-01 7.3024586e-02 3.5975783e-04 6.5722787e-03]
bn221018317 UNCERT [0.9061992  0.07968655 0.00123819 0.012876  ]
bn120716030 UNRELOC [9.8878342e-01 6.8209099e-04 3.5924322e-04 1.0175230e-02]
bn130628931 BELOWHZ [9.9967790e-01 2.1796502e-04 1.3945416e-05 9.0214024e-05]
bn211014070 UNCERT [9.9816376e-01 2.9325741e-04 4.7011170e-04 1.0729086e-03]
bn091022752 UNCERT [9.9997711e-01 2.0795505e-06 1.3184894e-05 7.5430803e-06]
bn230904491 UNCERT [1.0000000e+00 6.3237708e-09 1.4762901e-11 1.8833221e-08]
bn230913474 UNCERT [9.9998796e-01 3.1769578e-06 6.9301683e-07 8.1166236e-06]
bn2201157

In [ ]:
for i in interesting_events:
    n = output_event_names.index(i)
    print(output_event_names[n], output_event_types[n], output_event_preds[n])

bn170416538 UNRELOC [1.2407866e-27 1.0000000e+00 1.4984266e-29 1.2489068e-27]
bn240521839 UNCERT [5.8349408e-04 9.9410915e-01 1.9564922e-03 3.3509005e-03]
bn140704063 UNCERT [4.2431033e-08 9.9999797e-01 1.5661113e-06 5.2884269e-07]
bn211213088 UNCERT [0.0130097  0.97451824 0.0045687  0.00790345]
bn240513668 UNCERT [7.2027033e-04 9.2499679e-01 4.0147297e-02 3.4135591e-02]
bn190620772 UNRELOC [0.00480869 0.9882902  0.00168366 0.00521748]
bn150925345 UNCERT [1.5061334e-04 9.9971193e-01 7.7026561e-05 6.0399594e-05]
bn150424918 UNCERT [1.8542120e-05 9.9995625e-01 7.5921430e-06 1.7640161e-05]
bn190217091 UNRELOC [0.0262404  0.92678297 0.02075041 0.02622627]
bn170911787 UNCERT [3.5332126e-04 9.9942482e-01 1.0906932e-04 1.1288099e-04]
bn200518800 UNRELOC [2.1837843e-15 1.4993230e-12 8.2523964e-16 1.0000000e+00]
bn211029782 UNCERT [0.00148137 0.99227494 0.0028551  0.00338863]
bn240502886 UNCERT [1.06493610e-13 1.08709855e-13 1.47354868e-07 9.99999881e-01]
bn221029511 UNCERT [0.02686138 0.941458

In [ ]:
# False 514
# bn160114314
# [0, 0, 0, 0]
# [0.8616942  0.02299897 0.00460136 0.11070547]
# -----------------------------


# False
# bn230330654
# [0, 0, 0, 0]
# [0.94876885 0.02230632 0.00306564 0.02585921]
# -----------------------------

In [ ]:
# True
# bn080924766
# [1, 0, 0, 0]
# [0.01024451 0.93319297 0.02897306 0.02758948]
# -----------------------------

# True
# bn181212693
# [1, 0, 0, 0]
# [0.00209014 0.97003317 0.01413007 0.0137467 ]
# -----------------------------

# True
# bn080829790
# [1, 0, 0, 0]
# [0.00920342 0.9551996  0.00546318 0.03013378]
# -----------------------------

In [ ]:
print(len(event_names))

594


In [ ]:
print(event_names.index('bn181212693'))

41


In [ ]:
import requests
from bs4 import BeautifulSoup
import wget
import os
import subprocess
# wget -q -nH --no-check-certificate --cut-dirs=7 -r -l0 -c -N -np -R 'index*' -erobots=off --retr-symlinks https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/2012/bn120227725//current/
url =  'https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/2012/bn120227725//current/'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all links in the page
    links = soup.find_all('a')
    
    # Check for .rsp files
    rsp_files = [link.get('href') for link in links if link.get('href', '').endswith('.rsp')]

    print(rsp_files)

['glg_cspec_b0_bn120227725_v01.rsp', 'glg_cspec_b1_bn120227725_v01.rsp', 'glg_cspec_n0_bn120227725_v01.rsp', 'glg_cspec_n1_bn120227725_v01.rsp', 'glg_cspec_n2_bn120227725_v01.rsp', 'glg_cspec_n3_bn120227725_v01.rsp', 'glg_cspec_n4_bn120227725_v01.rsp', 'glg_cspec_n5_bn120227725_v01.rsp', 'glg_cspec_n6_bn120227725_v01.rsp', 'glg_cspec_n7_bn120227725_v01.rsp', 'glg_cspec_n8_bn120227725_v01.rsp', 'glg_cspec_n9_bn120227725_v01.rsp', 'glg_cspec_na_bn120227725_v01.rsp', 'glg_cspec_nb_bn120227725_v01.rsp', 'glg_ctime_b0_bn120227725_v01.rsp', 'glg_ctime_b1_bn120227725_v01.rsp', 'glg_ctime_n0_bn120227725_v01.rsp', 'glg_ctime_n1_bn120227725_v01.rsp', 'glg_ctime_n2_bn120227725_v01.rsp', 'glg_ctime_n3_bn120227725_v01.rsp', 'glg_ctime_n4_bn120227725_v01.rsp', 'glg_ctime_n5_bn120227725_v01.rsp', 'glg_ctime_n6_bn120227725_v01.rsp', 'glg_ctime_n7_bn120227725_v01.rsp', 'glg_ctime_n8_bn120227725_v01.rsp', 'glg_ctime_n9_bn120227725_v01.rsp', 'glg_ctime_na_bn120227725_v01.rsp', 'glg_ctime_nb_bn120227725_v